# Translate text to other languages

You will take the text chunks and questions that you have created in the previous notebook and translate them to other languages using Gemini.

Define the target language you want to translate to.

In [1]:
target_language = "arabic"

## Gemini

### Create the system prompt

Create a system prompt to teach the LLM how to perform the translation task.

In [2]:
system_prompt = f"""
## Task description

You are a world-class translator. Your task is to translate a given text into a specified target language. Don't do a literal word-for-word translation; instead, ensure that the translated text captures the original meaning, tone, and context.

### Important instructions:

- Don't add or remove any information from the original text.
- Maintain the original tone and style of the text.
- Return only the translated text without any additional commentary or explanations.

Here is the text, please translate it into {target_language}:
"""

### Create a Pydantic object

We use a Pydantic object to define the structure of the output we want from the LLM. This will force the LLM to return the output in a structured format that we can easily parse.

In [3]:
from pydantic import BaseModel


class TranslatedText(BaseModel):
    text: str

### Load the data

In [4]:
import json

prefix = "rog_strix_gaming_notebook_pc_unscanned_file_chunks"
text_chunk_file_path = f"../data/chunks/{prefix}.json"
questions_file_path = f"../data/question_answer_pairs/{prefix}_qa_pairs.json"

with open(text_chunk_file_path, "r") as f:
    text_chunks = json.load(f)

with open(questions_file_path, "r") as f:
    questions = json.load(f)

print(f"Loaded {len(text_chunks)} text chunks.")
print(f"Loaded {len(questions)} questions.")

Loaded 43 text chunks.
Loaded 377 questions.


### Use Gemini

In [5]:
from google import genai
from dotenv import load_dotenv

load_dotenv()
google_client = genai.Client()

Prepare to save the translated chunks and questions in the same direcotories as before but with a suffix indicating the target language. 

For example, if the target language is Arabic, the files will be saved as `chunks_arabic.json` and `chunks_qa_pairs_arabic.json`.

In [6]:
import time
from google.genai.errors import ServerError

from tqdm import tqdm

MAX_RETRIES = 5
INITIAL_DELAY_SECONDS = 2

for chunk in tqdm(text_chunks, total=len(text_chunks)):
    text = chunk["text_chunk"]

    delay = INITIAL_DELAY_SECONDS
    for attempt in range(MAX_RETRIES):
        try:
            response = google_client.models.generate_content(
                model="gemini-2.5-pro",
                # model="gemini-2.5-flash",
                contents=[
                    system_prompt,
                    text,
                ],
                config={
                    "response_mime_type": "application/json",
                    "response_schema": TranslatedText,
                    "max_output_tokens": 65_536,
                },
            )

            translated_text: TranslatedText = response.parsed  # type: ignore
            chunk["text_chunk"] = translated_text.text

            break

        except ServerError as e:
            if attempt == MAX_RETRIES - 1:
                print(f"FINAL ATTEMPT FAILED for a chunk. Error: {e}")
                # We are giving up on this chunk and moving to the next.
                # You could also 'raise e' here if you want your whole script to stop.
                break

            print(
                f"Model overloaded (503). Retrying in {delay} seconds... (Attempt {attempt + 1}/{MAX_RETRIES})"
            )

            time.sleep(delay)
            delay *= 2

        except Exception as e:
            print(f"An unexpected error occurred with a chunk: {e}")
            # Don't retry on unknown errors, just break and go to the next chunk
            break

  0%|          | 0/43 [00:00<?, ?it/s]

Model overloaded (503). Retrying in 2 seconds... (Attempt 1/5)
Model overloaded (503). Retrying in 4 seconds... (Attempt 2/5)
Model overloaded (503). Retrying in 8 seconds... (Attempt 3/5)


  0%|          | 0/43 [00:15<?, ?it/s]


KeyboardInterrupt: 

In [18]:
for question_object in tqdm(questions, total=len(questions)):
    question_text = question_object["question"]

    delay = INITIAL_DELAY_SECONDS
    for attempt in range(MAX_RETRIES):
        try:
            response = google_client.models.generate_content(
                # model="gemini-2.5-pro",
                model="gemini-2.5-flash",
                contents=[
                    system_prompt,
                    question_text,
                ],
                config={
                    "response_mime_type": "application/json",
                    "response_schema": TranslatedText,
                    "max_output_tokens": 65_536,
                },
            )

            translated_text: TranslatedText = response.parsed  # type: ignore
            question_object["question"] = translated_text.text

            break

        except ServerError as e:
            if attempt == MAX_RETRIES - 1:
                print(f"FINAL ATTEMPT FAILED for a chunk. Error: {e}")
                # We are giving up on this chunk and moving to the next.
                # You could also 'raise e' here if you want your whole script to stop.
                break

            print(
                f"Model overloaded (503). Retrying in {delay} seconds... (Attempt {attempt + 1}/{MAX_RETRIES})"
            )

            time.sleep(delay)
            delay *= 2

        except Exception as e:
            print(f"An unexpected error occurred with a chunk: {e}")
            # Don't retry on unknown errors, just break and go to the next chunk
            break

  0%|          | 1/377 [00:02<15:16,  2.44s/it]

Model overloaded (503). Retrying in 2 seconds... (Attempt 1/5)
Model overloaded (503). Retrying in 4 seconds... (Attempt 2/5)
Model overloaded (503). Retrying in 8 seconds... (Attempt 3/5)


  1%|▏         | 5/377 [00:24<22:05,  3.56s/it]  

Model overloaded (503). Retrying in 2 seconds... (Attempt 1/5)
Model overloaded (503). Retrying in 4 seconds... (Attempt 2/5)
Model overloaded (503). Retrying in 8 seconds... (Attempt 3/5)
Model overloaded (503). Retrying in 16 seconds... (Attempt 4/5)


  2%|▏         | 7/377 [01:01<1:02:12, 10.09s/it]

Model overloaded (503). Retrying in 2 seconds... (Attempt 1/5)


  3%|▎         | 11/377 [01:11<39:44,  6.51s/it] 


KeyboardInterrupt: 

### Save the translated data

In [ ]:
translated_text_chunk_file_path = f"../data/chunks/{prefix}_{target_language}.json"
translated_questions_file_path = f"../data/question_answer_pairs/{prefix}_qa_pairs_{target_language}.json"

with open(translated_text_chunk_file_path, "w") as f:
    json.dump(text_chunks, f, indent=2)

with open(translated_questions_file_path, "w") as f:
    json.dump(questions, f, indent=2)